In [7]:
import pandas as pd
from models.Hermen.model_BE2 import model_BE2
import numpy as np

FBS = pd.read_csv('models/Hermen/data/FBS.csv')
QC = pd.read_csv('models/Hermen/data/QC.csv').drop(columns=['Item']).rename(columns={'FBS_item': 'Item'})
RL = pd.read_csv('models/Hermen/data/RL.csv').dropna(subset=['GGI_code'])

QC = QC.groupby(['Item', 'GGI_code', 'Year', 'Area']).mean().reset_index()

In [23]:
FBS.Unit.unique()

array(['1000 persons', 'kcal/capita/day', '1000 tonnes', 'kg/capita/day'],
      dtype=object)

Use QC

In [8]:
# Aux functions, to improve. The idea is to run the model on the full database at once rather than spliiting all the stuff. Perharps using dataframe around
def pivot(df):
    return df.pivot(index=['Area', 'Year','Item'], columns='GGI_code', values='Value')

def df_to_dict(df):
    X = {}
    for code in df.columns:
        X[code] = df[code].fillna(0)
    return X

def FBS_QC_RL_to_X(FBS, QC, RL):
    
    
    X = {}
    
    FBS_dict = df_to_dict(pivot(FBS))
    QC_dict = df_to_dict(pivot(QC))
    RL_dict = RL[['Area', 'Year', 'GGI_code', 'Value']].pivot(index=['Area', 'Year'], columns='GGI_code', values='Value')
    RL_dict = df_to_dict(RL_dict)
    
    for dictionnary in [FBS_dict, QC_dict, RL_dict]:
        X.update(dictionnary)
    
            
    crop_group = FBS.dropna()[FBS.dropna()['group'] == 'crop'].Item.unique()
    animal_group = FBS.dropna()[FBS.dropna()['group'] == 'animal'].Item.unique()
    
    X['P'] = X['P'].dropna()
    X['crop_group'] = crop_group
    X['animal_group'] = animal_group

    X['FWPPi'] = 100
    X['FWCRi'] = 0
    X['CRfd'] = 1
    X['CD_corr'] = 1
    X['R_rate'] = 0

    
    # To clean up
    FDi = X['FDi'].reset_index('Item')

    # to check
    X['TCLD_baseline'] = X['TCLDi'].groupby(level=['Area', 'Year']).sum()
    
    X['FMi'] = X['FDi'].loc[:, :, crop_group] / X['FDi'].loc[:, :, crop_group].groupby(level=['Area', 'Year']).sum()
    
    X['P'] = X['P'].reset_index('Item', drop=True).drop_duplicates()
    

    return X


X = FBS_QC_RL_to_X(FBS, QC, RL)

In [16]:
__author__ = 'Hermen'
__status__ = 'Pending Validation'

"""
TO DO.
"""
from graphmodels.graphmodel import GraphModel
import pandas as pd


# FOOD DEMAND PER FOOD GROUP AGGREGATION

FDTi_nodes_bis = [
    {'type': 'input',
     'unit': 'x 1000 t',
     'id': 'FLOi',
     'name': 'Food losses per food group',
     },

    {'type': 'input',
     'unit': 'kg/capita/day',
     'id': 'FDKGi',
     'name': 'Kg food demand per day per food group',
     },
    {'type': 'input',
     'unit': 'kcal/capita/day',
     'id': 'FDKCi',
     'name': 'Kcal food demand per day per food group',
     },
    {'type': 'variable',
     'unit': '1',
     'id': 'SSRi',
     'name': 'Self-sufficiency ratio per food group',
     'in': ['FPi', 'FEi', 'FIi'],
     'computation': {'name': 'FPi / (FPi - FEi + FIi)',
                     'formula': lambda X: X['FPi'] / (X['FPi'] - X['FEi'] + X['FIi'])}
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FPi',
     'name': 'Food production per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FEi',
     'name': 'Food exports per food group',
     },

    {'type': 'input',
     'unit': '1000 t',
     'id': 'FIi',
     'name': 'Food imports per food group',
     },

    {'type': 'variable',
     'unit': '1000 t',
     'id': 'FDPi',
     'name': 'Total domestic food production per food group',
     'in': ['FDKGi', 'SSRi', 'FLOi', 'P'],
     'computation': {'name': '(FDKGi * P *1e3 * 365 + FLOi)  * SSRi',
                     'formula': lambda X: (X['FDKGi'] * 1e-9 * 365 * X['P'] * 1e3  + X['FLOi'] ) * X['SSRi']}  # to check
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FDi',
     'name': 'Feed demand per group',

     },
    {'type': 'variable',
     'unit': '1000 t',
     'id': 'OFi',
     'name': 'Other food demand',
     'in': ['SDi', 'NFDi', 'PDi', 'RDi', 'SVi'],
     'computation': {'name': 'SDi + NFDi + PDi + RDi + SVi',
                     'formula': lambda X: X['SDi'] + X['NFDi'] + X['PDi'] + X['RDi'] + X['SVi']}
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'SDi',
     'name': 'Seed demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'NFDi',
     'name': 'Non-food demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'PDi',
     'name': 'Processed demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'RDi',
     'name': 'Residual demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'SVi',
     'name': 'Stock variation per food group',
     },
    {'type': 'output',
     'id': 'FDTi',
     'name': 'Total food demand per food group',
     'unit': 'x 1000 tonne',
     'in': ['FDPi', 'OFi', 'FDi', 'SSRi'],
     'computation': {'name': 'SSRi * (OFi + FDi) + FDPi',
                     'formula': lambda X: X['SSRi'] * (X['OFi'] + X['FDi']) + X['FDPi']}
     },
    {'type': 'input',
     'unit': '1000 persons',
     'id': 'P',
     'name': 'Total population',
     },
]

FDTi_nodes = [
    {'type': 'variable',
     'unit': 'kcal/day',
     'id': 'TCDi',
     'name': 'Total calorie demand per food group',
     'in': ['P', 'FWi', 'FWCRi', 'FDKCi'],
     'computation': {'name': 'P*(FDKCi+FWi-FWCRi)',
                     'formula': lambda X:  X['P'] * (X['FDKCi'] + X['FWi'] - X['FWCRi']) * 1e3}
     },
    {'type': 'parameter',
     'unit': 'kcal/cap/day',
     'id': 'FWCRi',
     'name': 'Waste reduction policy consumption per food group',
     },
    {'type': 'parameter',
     'id': 'FWPPi',
     'unit': '%',
     'name': 'Waste reduction policy production per food group',
     },
    {'type': 'variable',
     'unit': 'kcal/cap/day',
     'id': 'FWi',
     'name': 'Food waste per food group',
     'in': ['FWPi_baseline', 'FWPPi'],
     'computation': {'name': 'FWPi_baseline * FWPPi',
                     'formula': lambda X: X['FWPi_baseline'] * X['FWPPi'] * 1e-2 / 365}
     },
    {'type': 'variable',
     'unit': 'kcal/cap/day',
     'id': 'FWPi_baseline',
     'name': 'Food waste baseline per food group',
     'in': ['FLOi', 'P', 'KKRi'],
     'computation': {'name': 'FLOi * KKRi / P',
                     'formula': lambda X: X['FLOi'] * X['KKRi'] / X['P'] * 1e-3}
     },
    {'type': 'input',
     'unit': 'x 1000 t',
     'id': 'FLOi',
     'name': 'Food losses per food group',
     },
    {'type': 'variable',
     'id': 'KKRi',
     'name': 'Kcal/1000 tons ratio per food group',
     'unit': 'Kcal/1000 tons',
     'in': ['FDKGi', 'FDKCi'],
     'computation': {'name': 'FDKGi / FDKCi',
                     'formula': lambda X: X['FDKCi'] / (X['FDKGi'] * 1e-6) }
     },
    {'type': 'input',
     'unit': 'kg/capita/day',
     'id': 'FDKGi',
     'name': 'Kg food demand per year per food group',
     },
    {'type': 'input',
     'unit': 'kcal/capita/day',
     'id': 'FDKCi',
     'name': 'Kcal food demand per day per food group',
     },
    {'type': 'input',
     'unit': 'capita',
     'id': 'P',
     'name': 'Total population',
     },
    {'type': 'variable',
     'unit': '1',
     'id': 'SSRi',
     'name': 'Self-sufficiency ratio per food group',
     'in': ['FPi', 'FEi', 'FIi'],
     'computation': {'name': 'FPi / (FPi - FEi + FIi)',
                     'formula': lambda X: X['FPi'] / (X['FPi'] - X['FEi'] + X['FIi'])}
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FPi',
     'name': 'Food production per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FEi',
     'name': 'Food exports per food group',
     },

    {'type': 'input',
     'unit': '1000 t',
     'id': 'FIi',
     'name': 'Food imports per food group',
     },

    {'type': 'variable',
     'unit': '1000 t',
     'id': 'FDi',
     'name': 'Total domestic food production per food group',
     'in': ['KKRi', 'SSRi', 'TCDi'],
     'computation': {'name': 'TCDi * SSRi / KKRi',
                     'formula': lambda X: X['TCDi'] * X['SSRi'] / X['KKRi'] *365}  # to check
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FDi_baseline',
     'name': 'Feed demand per group',

     },
    {'type': 'variable',
     'unit': '1000 t',
     'id': 'OFi',
     'name': 'Other food demand',
     'in': ['SDi', 'NFDi', 'PDi', 'RDi', 'SVi'],
     'computation': {'name': 'SDi + NFDi + PDi + RDi + SVi',
                     'formula': lambda X: X['SDi'] + X['NFDi'] + X['PDi'] + X['RDi'] + X['SVi']}
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'SDi',
     'name': 'Seed demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'NFDi',
     'name': 'Non-food demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'PDi',
     'name': 'Processed demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'RDi',
     'name': 'Residual demand per food group',
     },
    {'type': 'input',
     'unit': '1000 t',
     'id': 'SVi',
     'name': 'Stock variation per food group',
     },
    {'type': 'output',
     'id': 'FDTi',
     'name': 'Total food demand per food group',
     'unit': 'x 1000 tonne',
     'in': ['FDi', 'OFi', 'FDi_baseline', 'SSRi'],
     'computation': {'name': 'SSRi * (OFi + FDi_baseline) + FDi',
                     'formula': lambda X: X['SSRi'] * (X['OFi'] + X['FDi_baseline']) + X['FDi']}
     },
]

FDTi_model = GraphModel(FDTi_nodes)

# ANIMAL FEED DEMAND PER FOOD GROUP

FDi_nodes = [
    # FDTi from the previous function
    {'type': 'input',
     'unit': '1000 t',
     'id': 'FDTi_animal',
     'name': 'Total food demand per food group',

     },
    {'type': 'output',
     'unit': '1000 t',
     'id': 'FDi_crops',
     'name': 'Total animal feed demand per crop',
     'in': ['FMi', 'FD'],
     'computation': {'name': 'FMi * FD',
                     'formula': lambda X: X['FMi'] * X['FD']}
     },
    # Note1: The vector 'FMi' might contain animal-based feed (i.e. Milk) which caused a cycle error in my previous model; so we want to select only crops from the vector FMI
    {'type': 'parameter',
     'unit': '',
     'id': 'FMi',
     'name': 'Feed mix fraction',
     },
    # Note2: the CRfd is calculated for the year 2017; and assumed constant. For hungary the value of CRfd is: 1.58729526339088
    {'type': 'variable',
     'unit': '1000 t',
     'id': 'FD',
     'name': 'Total crop-feed demand',
     'in': ['TAFDi', 'CRfd'],
     'computation': {'name': 'SUM(TAFDi)/CRfd',
                     'formula': lambda X: X['TAFDi'].groupby(level=['Area', 'Year']).sum() / X['CRfd']}
     },
    # CRfd is calculated with model data from  & assumed constant for simulated years (for now)
    {'type': 'parameter',
     'unit': '',
     'id': 'CRfd',
     'name': 'Feed/forage distribution (correction)',
     },

    {'type': 'parameter',
     'unit': 'kg DM feed/ kg EW',
     'id': 'FCRi',
     'name': 'Feed conversion ratio',
     },
    {'type': 'variable',
     'unit': 'kg',
     'id': 'TAFDi',
     'name': 'Total animal feed demand per food group (animals)',
     'in': ['FCRi', 'FDTi_animal'],
     'computation': {'name': 'FRCi*FDTi_animal',
                     'formula': lambda X: X['FCRi'].multiply(X['FDTi_animal'], level='Item')
                    }
     }
]

FDi_crops_Model = GraphModel(FDi_nodes)

# FOOD DEMAND PER FOOD GROUP AGGREGATION with feed taken into account

def compute_FDTi_animal(X):
    X = X.copy()
    X['FDi_baseline'] = X['FDi']
    result = FDTi_model.run(X)['FDTi']
    return result.loc[(slice(None), slice(None), X['animal_group'])]


def compute_FDTi_crops(X):
    X = X.copy()
    #to fix
    X['FDi_baseline'] = pd.concat([X['FDi_crops'], pd.Series(0, index=X['FDTi_animal'].index)]).drop_duplicates()
    result = FDTi_model.run(X)['FDTi']
    return result.loc[(slice(None), slice(None), X['crop_group'])]


def compute_FDi_crops(X):
    X = X.copy()
    X['FDi_baseline'] = X['FDi']
    result = FDi_crops_Model.run(X)
    return result['FDi_crops']


full_FDTi_nodes = [
    {
        'id': 'food_balance_sheet',
        'name': 'All inputs (animals and crops)',
        'unit': '',
        'type': 'input',
    },
    {
        'id': 'FDTi_animal',
        'name': 'Total food demand per food group (animals)',
        'unit': '',
        'type': 'variable',
        'in': ['food_balance_sheet'],
        'computation': {'name': 'compute FDTi_animal', 'formula': lambda X: compute_FDTi_animal(X)}
    },
    {
        'id': 'FDi_crops',
        'name': 'Total domestic food production per food group (crops)',
        'unit': '',
        'type': 'variable',
        'in': ['food_balance_sheet', 'FDTi_animal', 'CRfd'],
        'computation': {'name': 'compute FDi_crops', 'formula': lambda X: compute_FDi_crops(X)}
    },
    {
        'id': 'FDTi_crops',
        'name': 'Total food demand per food group (crops)',
        'unit': '',
        'type': 'variable',
        'in': ['food_balance_sheet', 'FDi_crops'],
        'computation': {'name': 'compute FDTi_crops', 'formula': lambda X: compute_FDTi_crops(X)}
    },
    {
        'id': 'FDTi',
        'name': 'Total food demand per food group',
        'unit': '',
        'type': 'output',
        'in': ['FDTi_animal', 'FDTi_crops'],
        'computation': {'name': 'concatenate FDTi_crops and FDTi_animal', 'formula': lambda X: pd.concat([X['FDTi_animal'], X['FDTi_crops']])}
    },
    {
        'id': 'CRfd',
        'name': 'Feed/forage distribution (correction)',
        'unit': '',
        'type': 'parameter',
    },
]


model_full_FDTi = GraphModel(full_FDTi_nodes)


chCL_nodes = [
    # this is the value that is computed with the addition of animal feed (so the CRfd part)
    {'type': 'input',
     'id': 'FDTi',
     'name': 'Total food demand per food group',
     'unit': '1000 tons',
     },
    {'type': 'input',
     'unit': 'hg/ha',
     'id': 'CYi',
     'name': 'Crop yields per crop type',
     },
    {'type': 'input',
     'unit': 'ha',
     'id': 'TCLD_baseline',
     'name': 'Total crop land demand (baseline)',
     },
    {'type': 'parameter',
     'unit': '',
     'id': 'CD_corr',
     'name': 'Correction parameter crop demand',
     },
    {'type': 'variable',
     'id': 'TCLD',
     'name': 'Total cropland demand',
     'unit': 'ha',
     'in': ['FDTi', 'CYi', 'CD_corr'],
     'computation': {'name': 'sum(FDTi_crops / CYi) * CD_corr',
                     'formula': lambda X: (X['FDTi'].loc[(slice(None), slice(None), X['crop_group'])] / X['CYi']).groupby(level=['Area', 'Year']).sum() * X['CD_corr'] * 1e7}
                     
     },
    {'type': 'output',
     'id': 'chCL',
     'name': 'Change in cropland',
     'unit': '1000 ha',
     'in': ['TCLD', 'TCLD_baseline'],
     'computation': {'name': 'TCLD-TCLD_baseline',
                     'formula': lambda X: (X['TCLD'] - X['TCLD_baseline']) * 1e-5} # to check !!!
     }
]

chCL_model = GraphModel(chCL_nodes)


def FL(chCL, FL_t_minus_1, IL_t_minus_1):
    df = pd.concat([chCL, FL_t_minus_1, IL_t_minus_1],axis=1).dropna()
    df.columns = ['chCL', 'FL_t_minus_1', 'IL_t_minus_1']
    df['FL'] = df['FL_t_minus_1']
    
    df[(df.chCL > 0) & (df.IL_t_minus_1 - df.chCL) < 0]['FL'] += df['IL_t_minus_1'] - df['chCL']
    
    return df['FL']


def IL(chCL, IL_t_minus_1):
    df = pd.concat([chCL, IL_t_minus_1],axis=1).dropna()
    df.columns = ['chCL', 'IL_t_minus_1']
    df['IL'] = df['IL_t_minus_1'] - df['chCL']
    df[(df.chCL > 0) & (df.IL_t_minus_1 - df.chCL)<0] = 0
    return df['IL']


CH_IL_FL_nodes = [
    # from previous part
    {'type': 'input',
     'id': 'chCL',
     'name': 'Change in cropland',
     'unit': 'x 1000ha'
     },
    {'type': 'input',
     'unit': 'x1000 ha',
     'id': 'CL_t_minus_1',
     'name': 'Cropland t-1',
     },
    {'type': 'input',
     'unit': 'x1000 ha',
     'id': 'IL_t_minus_1',
     'name': 'Inactive land t-1',
     },
    {'type': 'input',
     'unit': 'x1000 ha',
     'id': 'FL_t_minus_1',
     'name': 'Forest land t-1',
     },
    {'type': 'output',
     'id': 'CL_t',
     'name': 'Cropland stock',
     'unit': 'x 1000ha',
     'in': ['chCL', 'CL_t_minus_1'],
     'computation': {'name': 'CL_t_minus_1 + chCL',
                     'formula': lambda X: X['chCL'] + X['CL_t_minus_1']}
     },
    {'type': 'output',
     'id': 'IL_t',
     'name': 'Inactive land stock',
     'unit': 'x 1000 ha',
     'in': ['chCL', 'IL_t_minus_1'],
     'computation': {'name': 'IL_computation(chCL, CL_t_minus_1) ',
                     'formula': lambda X: IL(X['chCL'], X['IL_t_minus_1'])}
     },
    {'type': 'output',
     'id': 'FL_t',
     'name': 'Forest land stock',
     'unit': 'x 1000ha',
     'in': ['chCL', 'FL_t_minus_1', 'IL_t'],
     'computation': {'name': 'FL_computation(chCL, FL_t_minus_1, IL_t)',
                     'formula': lambda X: FL(X['chCL'], X['FL_t_minus_1'], X['IL_t'])}
     },
]


model_CH_IL_FL = GraphModel(CH_IL_FL_nodes)

BE2_partial_nodes = [
    {'type': 'input',
     'unit': '1000 ha',
     'id': 'TLA',
     'name': 'Total land area',
     },
    {'type': 'input',
     'unit': '1000 ha',
     'id': 'FL_t',
     'name': 'Forest land stock',
     },
    {'type': 'input',
     'unit': '1000 ha',
     'id': 'IL_t',
     'name': 'Inactive land stock',
     },
    {'type': 'parameter',
     'unit': '%',
     'id': 'R_rate',
     'name': 'Rate of reforestation',
     },
    {'type': 'variable',
     'id': 'RF_land',
     'name': 'Reforestation of land',
     'unit': 'x 1000ha',
     'in': ['R_rate', 'IL_t'],
     'computation': {'name': 'R_rate * IL_t',
                     'formula': lambda X: X['R_rate'] * 1e-2 * X['IL_t']}
     },
    {'type': 'variable',
     'id': 'FL_RF',
     'name': 'Forest land stock after reforestation policy',
     'unit': 'x 1000ha',
     'in': ['RF_land', 'FL_t'],
     'computation': {'name': 'FL + RF_land',
                     'formula': lambda X: X['FL_t'] + X['RF_land']}
     },
    {'type': 'variable',
     'id': 'IL_RF',
     'name': 'Inactive land stock after reforestation policy',
     'unit': 'x 1000ha',
     'in': ['RF_land', 'IL_t'],
     'computation': {'name': 'IL_t - RF_land',
                     'formula': lambda X: X['IL_t'] - X['RF_land']}
     },
    {'type': 'output',
     'id': 'BE2',
     'name': 'Share of forest area to total land area',
     'unit': '%',
     'in': ['TLA', 'FL_RF'],
     'computation': {'name': '100 * (FL_RF / TLA)',
                     'formula': lambda X: 1e2 * X['FL_RF'] / X['TLA']}
     },
]


model_BE2_partial = GraphModel(BE2_partial_nodes)


model_BE2 = GraphModel(BE2_partial_nodes + CH_IL_FL_nodes + chCL_nodes + full_FDTi_nodes)#.run(X)

In [17]:
X['FDi_baseline'] = X['FDi']
res = FDTi_model.run(X)

In [18]:
res['FDTi'].loc['Hungary', 2017, ["Cereals - Excluding Beer",
                                 'Bovine Meat',
                                 'Mutton & Goat Meat',
                                 "Pigmeat", 
                                 'Poultry Meat', 
                                 'Meat, Other',
                                 'Starchy Roots',
                                 'Sugar & Sweeteners',
                                 'Pulses']]

Area     Year  Item                    
Hungary  2017  Bovine Meat                    30.562463
               Cereals - Excluding Beer    14019.854905
               Meat, Other                     6.486000
               Mutton & Goat Meat              1.362200
               Pigmeat                       469.766947
               Poultry Meat                  493.103660
               Pulses                         52.565465
               Starchy Roots                 402.041635
               Sugar & Sweeteners            754.207772
dtype: float64

In [19]:
(res['FDTi'] / res['FPi']).loc['France', 2017, slice(None)].dropna()

Area    Year  Item               
France  2017  Alcoholic Beverages    1.000118
              Animal fats            0.999949
              Apples and products    1.000146
              Barley and products    0.999943
              Beans                  0.993843
                                       ...   
              Vegetable Oils         0.999879
              Vegetables             1.000176
              Vegetables, Other      1.000026
              Wheat and products     1.000036
              Wine                   1.000014
Length: 70, dtype: float64

In [20]:
model_BE2 = GraphModel(BE2_partial_nodes + CH_IL_FL_nodes + chCL_nodes + FDTi_nodes)#.run(X)
res = model_BE2.run(X)

In [21]:
(res['FDTi'] / res['FPi']).loc['France', 2017, slice(None)].dropna()

Area    Year  Item               
France  2017  Alcoholic Beverages    1.000118
              Animal fats            0.999949
              Apples and products    1.000146
              Barley and products    0.999943
              Beans                  0.993843
                                       ...   
              Vegetable Oils         0.999879
              Vegetables             1.000176
              Vegetables, Other      1.000026
              Wheat and products     1.000036
              Wine                   1.000014
Length: 70, dtype: float64

In [ ]:
res['FDTi'].reset_index().Item.unique()

In [ ]:
res['TCLD'].loc['Hungary', 2017]

In [ ]:
comparison = pd.concat([res['TCLD'], res['TCLD_baseline']], axis=1)
comparison.columns= ['TCLD', 'TCLD_baseline']
comparison['CD_corr'] = comparison['TCLD'] / X['TCLD_baseline']

In [ ]:
comparison['CD_corr'].loc['Hungary', 2017]

In [ ]:
comparison = comparison[comparison.TCLD > 0]
comparison = comparison[comparison.CD_corr < 100]

In [ ]:
comparison['CD_corr'].hist()

In [ ]:
comparison

In [ ]:
comparison.plot(kind='scatter', x='TCLD', y='TCLD_baseline')

In [ ]:
res['TCLD_baseline'].loc['Hungary', 2017]

In [ ]:
RL = pd.read_csv('models/Hermen/data/RL.csv')#.dropna(subset=['GGI_code'])
RL =  RL.pivot(index=['Area', 'Year'], columns='Item', values='Value')

In [ ]:
 RL['Cropland'].loc['Hungary', 2017]

In [ ]:
TCLD_baseline = RL['Cropland']  - RL['Land under permanent crops']

In [ ]:
TCLD_baseline.loc['Hungary', 2017]

In [ ]:
(RL['Land under temporary crops'] + RL['Land under permanent crops']).dropna()

In [ ]:
(X['CL_t_minus_1'] / TCLD_baseline).loc[('Hungary', 2017)]

In [ ]:
TCLD = res['TCLD'] * 1e-3

In [ ]:
(res['TCLD'] / res['TCLD_baseline']).loc[('Hungary', 2017)]

In [ ]:
(TCLD / TCLD_baseline).loc[('Australia', 2017)]

In [ ]:
CD_corr  = (TCLD / TCLD_baseline).dropna()

CD_corr = CD_corr[CD_corr > 0]
CD_corr = CD_corr[CD_corr < 10]

In [ ]:
test = pd.concat([TCLD, TCLD_baseline], axis=1)
test.columns = ['TCLD', 'TCLD_baseline']
test['CD_corr'] = test['TCLD'] / test['TCLD_baseline']

test = test[test['CD_corr']>0]
test = test[test['CD_corr']<100]


In [ ]:
test.loc[('France', slice(None))]

In [ ]:
test.loc[('Germany', slice(None))]

In [ ]:
test.loc[('Germany', slice(None))]

In [ ]:
CD_corr.hist(bins=10)

In [ ]:
model_BE2.draw()

In [ ]:
(TCLD / TCLD_baseline).dropna().loc[('Germany', 2017)]

In [ ]:
(TCLD_baseline / X['TCLD_baseline'] * 1e3).dropna().loc[('Hungary', 2017)]

# crops 

In [ ]:
model = GraphModel(BE2_partial_nodes + CH_IL_FL_nodes + chCL_nodes + full_FDTi_nodes)
res = model.run(X)

In [ ]:
model.draw()

In [ ]:
res['TLA'].loc[('Hungary', 2017)]

In [ ]:
res['IL_RF'].loc[('Hungary', 2017)]

In [ ]:
res['FDTi'].loc[('Hungary', 2017)].loc['Poultry Meat']

In [ ]:
country = 'Hu'
year = 2017
def select_country_year(df, country, year):
    return df[(df.Area==country) & (df.Year == year)].reset_index(drop=True)

In [ ]:
X = FBS_QC_RL_to_X(select_country_year(FBS, country, year),
                   select_country_year(QC, country, year),
                   select_country_year(RL, country, year),)

In [ ]:
res = GraphModel(BE2_partial_nodes + CH_IL_FL_nodes + chCL_nodes +  full_FDTi_nodes).run(X)

In [ ]:
res['TCLD']

In [ ]:
X['TCLD_baseline']

In [ ]:
RL = pd.read_csv('models/Hermen/data/RL.csv')#.dropna(subset=['GGI_code'])


In [ ]:
RL.set_index(['Area', 'Year','Item']).loc[('France', 2017, 'Cropland')]

In [ ]:
RL.set_index(['Area', 'Year','Item']).loc[('France', 2017, 'Land under permanent crops')]

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
__author__ = 'Hermen'
__status__ = 'Pending Validation'

"""
TO DO.
"""
from graphmodels.graphmodel import GraphModel

CD_corr_nodes = [
    {'type': 'input',
     'unit': '1000 tons',
     'id': 'FDTi',
     'name': 'Total food demand per food group',
     },
    {'type': 'input',
     'unit': 'hg/ha',
     'id': 'CYi',
     'name': 'Crop yields per crop type'},
    # Unit conversion factor 1e4 (ha -> 1000 ha, hg -> 1000 tons)
    {'type': 'variable',
     'unit': '',
     'id': 'TCLD',
     'name': 'Total crop land demand',
     'in': ['FDTi', 'CYi'],
     'computation':{'name': 'FDTi * CYi', 'formula': lambda X: 1e4 * (X['FDTi'] / X['CYi']).sum()}
     },
    {'type': 'input',
     'unit': '1000 ha',
     'id': 'LUi_baseline',
     'name': 'Land use per group'
     },
    {'type': 'variable',
     'unit': 'ha',
     'id': 'TCLD_baseline',
     'name': 'Total crop land demand (baseline)',
     'in': ['LUi_baseline'],
     'computation': {'name': 'LU_Cropland - LU_Land_under_permanent_crops',
                     'formula': lambda X: X['LUi'].loc['Cropland'] - X['LUi'].loc['Land under permanent crops']}
     },
    {'type': 'output',
     'unit': '',
     'id': 'CD_corr',
     'name': 'Correction parameter crop demand',
     'in': ['TCLD', 'TCLD_baseline'],
     'computation': {'name': 'TCLD_baseline / TCLD',
                     'formula': lambda X: X['TCLD_baseline'] / X['TCLD']}
     },
]

GraphModel(CD_corr_nodes).draw()
